In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas
import numpy as np

#read data csv
expression_data = pandas.read_csv("GSE89843_TEP_Count_Matrix.txt", header=0, sep=" ")
expression_data = expression_data.iloc[:,1:].T

patient_data = pandas.read_csv("PatientData.csv", header=0, sep="\t")
patient_group = patient_data.loc[:,["GEO ID (GSE89843)", "Classification group"]]
patient_group.set_index("GEO ID (GSE89843)", inplace=True)

merged_data = pandas.concat([expression_data, patient_group], axis=1, join="inner")
recoded_y = [1 if i == "NSCLC" else 0 for i in merged_data["Classification group"]]
merged_data.drop(columns=["Classification group"], inplace=True)

in_data = merged_data.to_numpy()
out_data = np.array(recoded_y)

#test-train split
x_train, x_test, y_train, y_test = train_test_split(in_data, out_data, test_size=0.25, random_state=333)

#fit random forest
model = RandomForestClassifier()
model.fit(x_train, y_train)

#evaluate random forest
print(model.score(x_test, y_test))

0.8153846153846154
